#### 月报数据处理

###### 取数函数思路：对单人各周轮询取数，各周dataframe放入list datadocker里面，传给传递函数
###### 传递函数思路：逐个将每人datadocker list里各周dataframe合并，传递到alldatadocker 这个dataframe里


In [ ]:
import numpy as np
import pandas as pd
import os
import re
from numpy import nan
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import platform


In [ ]:
# ----------------------------------------取数函数-------------------------------------
def dataacs(path,name):  
    # print("正在处理"+name[0]+"的周报")
    # docker作为回传数据容器
    datadocker = list()

    # sht_ai 页签数
    sht_ai = len(pd.read_excel(path,header=None,sheet_name=None).keys())

    # sht_name = list(dict_keys) 页签名字列表
    sht_aname = list(pd.read_excel(path,header=None,sheet_name=None).keys())

    # sht_aname_index 取出周工作表页签名含有周工作的bool list
    sht_aname_index = list()
    for i in sht_aname:
        if "周工作" in i: sht_aname_index.append(True)
        else:sht_aname_index.append(False)
    t = [i for i in range(0,sht_ai)]
    sht_real_ai = np.array(t)[np.array(sht_aname_index)]

    for sht_i in sht_real_ai:
        sht_value = pd.read_excel(path,header=None,sheet_name=sht_i,engine='openpyxl')
        # df[bool list] 筛选行数
        first_col_list  = sht_value.iloc[:,0]
        frnum = first_col_list[first_col_list.str.contains('任务编号',na=False)].index[0] + 1
        lrnum = first_col_list[first_col_list.str.contains('小计',na=False)].index[0] 

        rnum = lrnum - frnum  #行数(contain total)

        # df[bool list] 筛选列数
        first_row_list  = sht_value.iloc[1,:]
        cnum = first_row_list[first_row_list.str.contains('统计',na=False)].index[0]

        # 数据框提取
        worktemp = sht_value[frnum-1:lrnum+1]
        worktemp = worktemp.iloc[:,0:(cnum+1)]
        rowname = worktemp.iloc[0]
        worktemp = worktemp.rename(columns=rowname).iloc[1:]
        worktemp.columns = [*worktemp.columns[:-1], '项目投入统计']
        worktemp = worktemp.reset_index(drop=True)


        # 第一个‘星期一‘到‘项目投入统计‘ 列数
        t = worktemp.columns.to_frame()[0].reset_index(drop=True)
        sum_st_col = t[t.str.contains('星期一',na=False)].index[0]   
        sum_sp_col = cnum
        daynum = sum_sp_col - sum_st_col


#
        # # 数据矫正：重新输出项目投入统计列数据
        worktemp['项目投入统计'] = worktemp.iloc[:,sum_st_col:sum_sp_col].sum(axis=1)
        # if (worktemp['项目投入统计'][0:-1].sum()) == (worktemp['项目投入统计'].iloc[-1]):
        #     # print('项目投入统计正确')
        #     pass
        # # #工作天未自增1h
        # # elif ((worktemp['项目投入统计'].iloc[-1]) - worktemp['项目投入统计'][0:-1].sum()) == daynum:
        # #     # worktemp['项目投入统计'].iloc[:-1] = worktemp['项目投入统计'].iloc[:-1] + (daynum/rnum)
        # #     # worktemp['项目投入统计'].iloc[-1] = worktemp['项目投入统计'].iloc[-1] + daynum
        # #     print(name[0]+'项目投入统计数值在第'+ str(sht_i) +'周出错（加和原因）')
        # else:
        #     # openpyxl 引擎可以读取公式数值
        #     print(name[0]+'项目投入统计数值在第'+ str(sht_i) +'周出错（填报原因）')  
        #     # xlrd 引擎无法读取公式数值 现在用于debug执行到所到问题excel页签
        #     # print(name[0]+'在第'+ str(sht_i) +'周出错')


        # # 取得数据
        # 清洗数据框
        worktemp_final = worktemp.iloc[:-1,:] #删除 小计 行
        worktemp_final = worktemp_final.loc[:,['任务分类','项目名称']] #取出相应列      ###项目code可以一并从这里取出来  

        worktemp_final['项目用时统计'] = worktemp.iloc[:,-1] #统计用时统一名称


        # 前2列 空单元格取上单元格值 （excel合并问题处理）
        if ((worktemp_final.iloc[:,0:2].isnull().sum().sum())!= 0):                     #判断前两列是否存在空值                                 |仍有逻辑漏洞
                                                                                        #    因为下面删除0行后空白列取上面的值可能出现逻辑错误     | 当前两列首航为空值
                                                                                        # -----------------------------列空白处理在这里完成     | 第一个值取值溢出
            #去除列的空值
            # judge_l = list()                          #每列空值所在行数
            # for z in range(0,2):                      #按列循环
            #     for i in range(0,len(worktemp_final.iloc[:,z])):    #控制所在行数确认  error
            #         if (pd.isnull(worktemp_final.iloc[:,z][i])):
            #             judge_l.append(i)
            #     for j in judge_l:                      #每列空值循环向上取值
            #         worktemp_final.iloc[j,z] = worktemp_final.iloc[j-1,z]

            for j in range(0,2):                                    #列
                for i in range(0,len(worktemp_final.iloc[:,j])):    #行
                    if(pd.isnull(worktemp_final.iloc[:,j][i])):
                        worktemp_final.iloc[i,j] = worktemp_final.iloc[i-1,j]
        
        #  去除尾列为0的行
        t = worktemp_final.iloc[:,-1]                                                  ##
        t = t.map(lambda x: (np.isnan(x)) or (x is None) or (x == " ") or (x == 0))     ##去除统计用时为0的工作任务
        worktemp_final = worktemp_final[~(t)]                                          ##
        worktemp_final = worktemp_final.rename({'项目用时统计':'W'+str(sht_i)},axis = 1)

        #当某周为空白时,删除改dataframe
        if (worktemp_final.shape[0] != 0):
            #回传真实数据
            datadocker.append(worktemp_final)

    return datadocker


In [ ]:
# ----------------------------------------传递函数-------------------------------------
# pd to excel 会删除原有数据 且指定插入   故需要将插入数据中间继承 再一次性导入
def container(name,datadocker,alldatadocker):
    datadockertemp = list()
    df = pd.DataFrame()
    # 暂时合并任务相关的两列 并 将任务作为列名方便concat合并当月所有任务用时(最后还需转置回来)

    # -------------------------------合并处理在这里完成(转置做法_old) -----------------------------------
    # for i in datadocker:
    #     temp = pd.DataFrame()
    #     temp["任务"] = i['任务分类'] + '_' + i['项目名称']
    #     temp['投入用时'] = i['项目用时统计']                      # 项目用时统计 列名已经在 取数函数 97行中取消
    #     temp = temp.groupby(['任务']).sum()                                  
    #     temp = temp.T
    #     # temp = temp.reset_index(drop=True)
    #     datadockertemp.append(temp)
    # df = pd.concat(datadockertemp) #员工当月数据以任务为列名合并（省去任务不同需要增加行的代码逻辑，任务不同自增一列）
    # df = df.T                      #转置回正常统计维度 (周为列)                       |项目投入(第一周)|项目投入(第二周)|...| ... |
    # df = df.reset_index()          #                                              |任务|项目投入(第一周)|项目投入(第二周)|...| ... |
    # df['员工'] = name[0]            #                                              |任务|项目投入(第一周)|项目投入(第二周)|...| ... |员工|
    # df = pd.concat([df, df['任务'].str.split('_', expand=True)], axis=1)#          |任务|项目投入(第一周)|项目投入(第二周)|...| ... |员工|0|1|   (0，1分别为任务分类和任务名称)
    # df_final = pd.concat([df.iloc[:,-3:],df.iloc[:,1:-3]],axis=1)       #          重新对列进行排序

    # -------------------------------合并处理在这里完成(concat/merge/join做法_new) -----------------------------------
    df_temp = pd.concat(datadocker,axis=0)
    df_temp = df_temp.groupby(['任务分类','项目名称'],as_index = False).sum()   #####  项目code加入后需要在这里加以下groupby list
    df_temp = df_temp.reset_index(drop=True)
    df_temp['员工'] = name[0]
    df_temp = pd.concat([df_temp.iloc[:,-1],df_temp.iloc[:,:-1]],axis=1) 
    alldatadocker = alldatadocker.append(df_temp)
    return alldatadocker

In [ ]:
# ----------------------------------------写入函数-------------------------------------
def datawritten(path_save,alldatadocker):
    alldatadocker.to_excel(path_save,index = False)


In [ ]:
# ------------------------------------ 主程序入口 ------------------------------------ CH
def main():
    #------------全局变量-------------------------------------------------
    #将文件夹中所有excel路径存入list   
    plat = platform.system().lower()
    p = os.getcwd()
    if plat == 'windows':
        p = p.replace('\\','/')
    fl = os.listdir('./input/itest')                                  #CH
    # fl = os.listdir('./input/'+str(datetime.now().year)[2:4]+'M'+str(datetime.now().month-1))  
    l = [i for i in fl if '工作周报' in i]                            #CH
    l = [p+"/input/itest/"+i for i in l]                              #CH
    # l = [p+"/input/"+str(datetime.now().year)[2:4]+'M'+str(datetime.now().month-1)+'/' + i for i in l]  


    #输出excel的保存位置
    path_save = './output/otest/汇总.xlsx'                            #CH
    # path_save = './output/汇总'+str(datetime.now().year)[2:4]+'_'+str(datetime.now().month-1)+'.xlsx' 

    alldatadocker = pd.DataFrame()
    #---------------------------------------------------------------------

    #循环从excel所有人周工作表中提取明细数据
    for g in l:
        path = g
        name = re.findall('周报_(.*)_2',path)                        #CH
        name = [i.strip() for i in name]
        datadocker = dataacs(path,name)
        alldatadocker = container(name,datadocker,alldatadocker)

    alldatadocker = alldatadocker.reset_index(drop=True)
    l = list(alldatadocker.columns)
    l[1],l[2] = '任务分类','任务'
    for i in range(3,len(l)):
        l[i] = 'Week'+str(i-2)
    alldatadocker.columns = [x for x in l]
    # df = df.replace(np.nan,0)
    alldatadocker = alldatadocker.fillna(0)
    alldatadocker['总人天'] = np.around((alldatadocker.iloc[:,3:].sum(axis=1))/7,2)
    alldatadocker
    datawritten(path_save,alldatadocker)
    return alldatadocker
    
df0 = main()


In [ ]:
# 删除所有变量
# %reset 

In [ ]:
# import subprocess, sys

# opener = "open" if sys.platform == "darwin" else "xdg-open"
# subprocess.call([opener, './'])


#### 月报分析数据展示

In [ ]:
df0

In [ ]:
# 23年法定人天  & 当前月法定人天        
month = datetime.now().month
mdlist = [18,20,23,	19,	21,	21,	21,	23,	20,	19,	22,	21]
mdinlaw = mdlist[month-1]
mdinlaw

In [ ]:
# 人员有效利用率 前8
df2 = df0[df0['任务分类'].isin(['建设','运维','营销'])].pivot_table(index=['员工'],aggfunc={'总人天': np.sum},values=['总人天']).reset_index()
df2['有效利用率_orgin'] = df2['总人天']/mdinlaw
df2['有效利用率'] = df2['有效利用率_orgin'].apply(lambda x : format(x, '.2%'))
df2 = df2.loc[:,['员工','有效利用率','总人天']]
df2 = df2.rename(columns={'总人天':'有效总人天'})
df2['有效利用率_orgin'] = df2['有效总人天']/mdinlaw
df2

In [ ]:
# 人员利用率 前8
df1 = df0.pivot_table(index=['员工'],aggfunc={'总人天': np.sum}).reset_index()
df1['法定总人天'] = mdinlaw 
df1['利用率_orgin'] = df1['总人天']/mdinlaw
df1['利用率'] = df1['利用率_orgin'].apply(lambda x : format(x, '.2%'))

# df1 = df1[['员工','人员利用率','总人天']]
df1 = df1.loc[:,['员工','利用率','总人天']]
df1 = df1.merge(df2.iloc[:,[0,2]],how='left',on='员工')
# df1.join(df2,lsuffix='',rsuffix='_rsuffix').iloc[:,[0,1,2,5]]
# pd.concat([df1,df2],axis=1).iloc[:,[0,1,2,5]]

df1['利用率_orgin'] = df1['总人天']/mdinlaw
df1.head(8)

In [ ]:
# 任务分类占比
df6 = df0.pivot_table(index=['任务分类'],aggfunc={'总人天': np.sum},values=['总人天']).reset_index()
df6['占比'] = df6['总人天']/(df6['总人天'].sum())
df6 = df6.iloc[:,[0,2]].sort_values(by=['占比'],ascending=False)
df6.iloc[:,0] = df6.iloc[:,0].apply(lambda x: x+'总人天')
df6.iloc[:,1] = df6.iloc[:,1].apply(lambda x:format(x,'.2%'))
df6

In [ ]:
# 任务分类投入人天
df7 = df0.pivot_table(index=['任务分类'],aggfunc={'总人天': np.sum},values=['总人天']).reset_index()
df7 = df7.iloc[:,[0,1]].sort_values(by=['总人天'],ascending=False)
df7['任务分类'] = df7['任务分类'] + '总人天'
df7

In [ ]:
# 打印关键指标 
print(format(datetime.now(),'%y%m'))

# 人数
print('总人数： ' + str(len(df0.iloc[:,0].unique())))

# 总人天
print('总人天： ' + str(np.around(df1.iloc[:,2].sum(),2)))

# 有效总人天
print('有效总人天： ' + str(np.around(df2.iloc[:,2].sum(),2)))

# 人均利用率
print('人均利用率' + format(np.mean(df1['利用率_orgin']),'.2%'))

# 人均有效利用率
print('人均有效利用率' + format(np.mean(df2['有效利用率_orgin']),'.2%'))


In [ ]:
# 建设项目投入人天 前8
df3 = df0[df0['任务分类'] == '建设']
df3 = df3.loc[:,['任务','总人天']].groupby('任务').sum().sort_values(by=['总人天'],ascending=False)
df3.head(8)

In [ ]:
# 建设项目投入人数 前8
df4 = df0[df0['任务分类'] == '建设']
df4 = df4.loc[:,['任务','员工']]
df4['员工'] = 1
df4 = df4.loc[:,['任务','员工']].groupby('任务').sum().sort_values(by=['员工'],ascending=False)
df4.head(8)

In [ ]:
# 系统运维投入人天 前8
df5 = df0[df0['任务分类'] == '运维']
df5 = df5.loc[:,['任务','总人天']].groupby('任务').sum().sort_values(by=['总人天'],ascending=False)
df5.head(8)

In [ ]:
# 系统运维投入人数 前8
df5 = df0[df0['任务分类'] == '运维']
df5 = df5.loc[:,['任务','员工']]
df5['员工'] = 1
df5 = df5.loc[:,['任务','员工']].groupby('任务').sum().sort_values(by=['员工'],ascending=False)
df5.head(8)

In [ ]:
# 组别维度表

groupdimension = pd.DataFrame({
    '员工' : ["朱苏明",   "符芳恺",	"陆小兰",	"董乐航",	"耿凡舒",	"李国靖",	"徐勇",	"刘攀",	"何远",	"卢绍松",	"廖美联",	"黎庆奋",	"陈其达",	"廖武燊",	"张劢",	"李奕信",	"吕光源",	"翁圳滨",	"冯求四",	"帅泉泉",	"许伟兴",	"韦庆生",	"黄国杰",	"滕兆悟",	"黄嘉杰",	"毛裕青",	"胡志忠",	"王超",	"邓承熹",	"陈亮",	"邱文杰",	"罗远",	"江忠",	"李霁恒",	"谭文辉",	"王可珂",	"揭俊娟",	"何新辉",	"周刚",	"史少龙",	"赵翠旺",	"李鹏"],
    'group' : ["公司治理",	"公司治理",	"公司治理",	"公司治理",	"公司治理",	"公司治理",	"物资供应",	"物资供应",	"物资供应",	"物资供应",	"物资供应",	"销售物流",	"销售物流",	"销售物流",	"销售物流",	"销售物流",	"数据应用",	"数据应用",	"数据应用",	"数据应用",	"新业务",	"新业务",	"新业务",	"通用技术",	"通用技术",	"通用技术",	"通用技术",	"基础设施",	"基础设施",	"基础设施",	"基础设施",	"基础设施",	"基础设施",	"基础设施",	"电商",	"电商",	"电商",	"电商",	"电商",	"电商",	"电商",	"电商"],
    'YN' : [1,	1,	1,	1,	1,	0,	1,	1,	1,	1,	1,	1,	1,	1,	1,	0,	1,	1,	1,	1,	1,	1,	1,	1,	1,	0,	1,	1,	1,	1,	1,	1,	1,	1,	1,	0,	1,	1,	1,	1,	1,	1]})
groupdimension

In [ ]:
groupdimension.values.tolist()

In [ ]:
groupdimension.groupby('group').sum()

In [ ]:
# 组别人数
df8 = groupdimension.groupby('group').sum().reset_index()
df8



df8

In [ ]:
# 业务组利用率
df9 = df0.merge(groupdimension,how='left',on='员工')
df9 = df9.loc[:,['group','总人天']]
df9 = df9.groupby(by='group').sum()
df9 = df9.merge(df8,how='left',on='group')
df9['tmd'] = df9['YN']*mdinlaw
df9['groupusingrate'] = df9['总人天']/df9['tmd']
df9 = df9[['group','groupusingrate']].sort_values(by='groupusingrate',ascending=False)
df9['groupusingrate'] = df9['groupusingrate'].apply(lambda x: format(x,'.2%'))
df9

In [ ]:
# 业务组有效利用率
df10 = df0[df0['任务分类'].isin(['建设','运维','营销'])].merge(groupdimension,how='left',on='员工')
df10 = df10.loc[:,['group','总人天']]
df10 = df10.groupby(by='group').sum()
df10 = df10.merge(df8,how='left',on='group')
df10['tmd'] = df10['YN']*mdinlaw
df10['groupusingrate'] = df10['总人天']/df10['tmd']
df10 = df10[['group','groupusingrate']].sort_values(by='groupusingrate',ascending=False)
df10['groupusingrate'] = df10['groupusingrate'].apply(lambda x: format(x,'.2%'))
df10

#### 月报ppt处理

In [ ]:
text_runs = []
from pptx import Presentation
ppt = Presentation('./润丰智慧-产品中中心管理月报（2月月会）-V1.7.pptx')
len(ppt.slides)
slide = ppt.slides[0]
shape = slide.shapes[0]
shape.text = '就这月报'

ppt.save('./就这月报.pptx')